In [ ]:
# 将 llm_toy/src 加入 sys.path（兼容多启动位置）
import sys
from pathlib import Path

def _add_src_path():
    candidates = [
        Path.cwd() / 'llm_toy' / 'src',           # 在项目根启动Jupyter
        Path.cwd() / 'src',                        # 在 llm_toy 目录启动
        Path.cwd().parent / 'llm_toy' / 'src',
        Path.cwd().parent / 'src',
    ]
    # 向上回溯几层尝试
    for base in list(Path.cwd().parents)[:3]:
        candidates.append(base / 'llm_toy' / 'src')
        candidates.append(base / 'src')
    for p in candidates:
        if (p / 'model.py').exists() and (p / 'utils.py').exists():
            sys.path.append(str(p.resolve()))
            print('已添加src路径:', p.resolve())
            return str(p.resolve())
    print('警告：未找到 llm_toy/src，请手动添加路径或调整工作目录。')
    return None

SRC_PATH = _add_src_path()


# 04 微调入门：在指令风格数据上做小规模 Fine-tuning

本Notebook展示如何对预训练的 GPT-2 做小规模 Fine-tuning，使其更贴近我们的小任务格式（如问答/指令）。

注意：示例数据量极小，目的在于跑通流程与理解方法，不追求效果。

In [ ]:
import os
from pathlib import Path
import torch
from torch.utils.data import DataLoader
import sys
# 路径已在开头插入cell中处理

from model import SimpleGPTModel
from trainer import LLMTrainer, SimpleDataset

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device


## 构造指令风格数据

我们使用极小的中文Q&A样例。实际项目请替换为更大、更干净的数据集。


In [ ]:
pairs = [
    ('问：什么是Transformer？', '答：一种基于Self-Attention的神经网络架构，用于序列建模。'),
    ('问：什么是Fine-tuning？', '答：在预训练模型基础上，用下游任务数据做进一步训练。'),
    ('问：什么是Tokenizer？', '答：把文本切分为tokens的工具，如BPE、WordPiece等方法。'),
    ('问：如何控制生成？', '答：可以调整temperature、top-k、top-p等采样参数。'),
]

# 拼接成统一输入格式
texts = [f'{q}\n{a}' for q, a in pairs]
len(texts), texts[0]


## 初始化模型与Tokenizer


In [ ]:
simple = SimpleGPTModel(model_name='gpt2')
tokenizer = simple.tokenizer
model = simple.model.to(device)

# 可选：冻结大部分层，只训练输出层或最后几层（示例）
for name, param in model.named_parameters():
    param.requires_grad = False
# 仅解冻最后两层block与lm_head（简化演示）
for block in model.transformer.h[-2:]:
    for p in block.parameters():
        p.requires_grad = True
for p in model.lm_head.parameters():
    p.requires_grad = True

sum(p.requires_grad for p in model.parameters()), sum(1 for _ in model.parameters())


## 构建Dataset/DataLoader并训练

- 为了演示稳定，batch小、step少、epoch=1
- 在CPU上也能跑通（速度较慢）


In [ ]:
ds = SimpleDataset(texts=texts, tokenizer=tokenizer, max_length=96)
dl = DataLoader(ds, batch_size=2, shuffle=True)
trainer = LLMTrainer(model=model, tokenizer=tokenizer, device=str(device))
from pathlib import Path as _P2
_cwd = _P2.cwd()
save_dir = str((_cwd/'llm_toy'/'checkpoints'))
trainer.train(train_dataloader=dl, val_dataloader=None, num_epochs=1, learning_rate=1e-4, save_dir=save_dir)


## 生成：观察风格贴合

尝试提供一个新的问题前缀，看看模型是否以“答：”的风格续写。


In [ ]:
prompt = '问：如何理解Self-Attention？'
ids = tokenizer.encode(prompt, return_tensors='pt').to(device)
with torch.no_grad():
    gen = model.generate(ids, max_length=80, do_sample=True, temperature=0.8, pad_token_id=tokenizer.eos_token_id)
print(tokenizer.decode(gen[0], skip_special_tokens=True))


## 思考与练习

- 不冻结参数，完整微调，与部分冻结对比效果
- 增加样本数量，观测风格拟合是否更明显
- 尝试加入英文指令，混合多语言场景
